# This is an LSTM on the last word of a stream

In [2]:
from __future__ import print_function
import os
import pickle
import numpy as np
import pandas as pd
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.optimizers import RMSprop
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN 5004)

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/')

In [4]:
w2indx = pickle.load(open('./0-output/LSTMDQN_Dic_token2id.pkl', 'rb'))
indx2w = pickle.load(open('./0-output/LSTMDQN_Dic_id2token.pkl', 'rb'))

In [5]:
len(w2indx), len(indx2w)

(239692, 239692)

In [15]:
qdf = pd.read_csv('./0-output/aurora_nuggets_numtext.csv', nrows=56)

In [16]:
adf = pd.read_csv('./0-output/2012_aurora_shooting_first_sentence_numtext.csv', nrows=50000)

In [17]:
rans = np.random.random((adf.shape[0],1)) >0.9
rans2 = np.random.random((adf.shape[0],1)) >0.9
rans3 = (rans==False)*rans2
X_train, X_test = adf.values[rans], adf.values[rans3]

In [18]:
X_train = np.asarray([ [int(j) for j in i.split(" ")] for i in X_train])
X_test = np.asarray([ [int(j) for j in i.split(" ")] for i in X_test])

In [19]:
X_train.shape, X_test.shape

((1221,), (1090,))

In [25]:
emb_dim = 10
batch_size = 32

# Y_train = np.random.choice([0,1], size=(len(X_train),1))
# Y_test = np.random.choice([0,1], size=(len(X_test),1))
Y_train = np.random.random(size=(len(X_train),1))
Y_test = np.random.random(size=(len(X_test),1))

In [21]:
# # Taking the last element of X_train[i] as our outcome
# Y_train = np.asarray([i[-1] for i in X_train]) 
# Y_test = np.asarray([ i[-1] for i in X_test])

# # Removing the last element of X_train[i]
# X_train = np.asarray([i[:-1] for i in X_train]) 
# X_test = np.asarray([i[:-1] for i in X_test] )

In [26]:
# print([x for x in X_train[0]])

In [27]:
print ([indx2w[x] for x in X_train[0] if x != 0])

[u'Us', u'Pause', u's', u'and', u'of', u'young', u'adults', u'for', u'the', u'of', u'lives']


In [28]:
len(np.unique(X_train)), len(np.unique(Y_train))

(1185, 1221)

In [18]:
# print(indx2w[Y_train[0]])

In [29]:
maxlen=40

In [30]:
# RNN for the regular embedings
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
# The pad sequences stuff seems suspicious
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

1221 train sequences
1090 test sequences
Pad sequences (samples x time)
X_train shape: (1221, 40)
X_test shape: (1090, 40)


In [48]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(indx2w), emb_dim)) # , dropout=0.2))
model.add(LSTM(100))
model.add(Dense(2))
model.add(Activation('relu'))
model.add(Dense(1))

# try using different optimizers and different optimizer configs

optimizer = RMSprop(lr=0.01)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

print('Train...')
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=5,
          validation_data=(X_test, Y_test), verbose=0)

# Running the predictions
out = model.evaluate(X_test, Y_test, batch_size=batch_size)
ypred_tst = model.predict(X_test)

Build model...
Train...
1090/1090 [==============================] - 1s     


In [51]:
out

[0.086311604424354135, 0.0]